In [ ]:
#Quantum Neural Network for MNIST Classification (0 vs 1)

import pennylane as qml
from pennylane import numpy as np
import torch 
import torchvision 
from torchvision import transforms 
from torch.utils.data import DataLoader, Subset





: 